In [9]:
import time
import matplotlib.pyplot as plt
import numpy as np
import math as mt
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from konlpy.tag import Hannanum, Mecab
from hangul_utils import split_syllables, join_jamos
from tqdm.auto import tqdm
from kiwipiepy import Kiwi
import re
from pykospacing import Spacing
#from eunjeon import Mecab
han = Hannanum()
mec = Mecab()

number = 100

# 초성 리스트. 00 ~ 18
CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
# 중성 리스트. 00 ~ 20
JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
# 종성 리스트. 00 ~ 27 + 1(1개 없음)
JONGSUNG_LIST = ['_', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

#선어말 어미
EP = [
    [['ㅈ','ㅓ','ㄴ','ㅡ','ㄴ'],['ㄴ','ㅏ','ㄴ','ㅡ','ㄴ']], # 저는 -> 나는
    [[' ','ㅈ','ㅓ','ㄴ',' '],['ㄴ','ㅏ','ㄴ','ㅡ','ㄴ']], # 전 -> 난 
    [['ㅇ','ㅓ','ㅇ','ㅛ',],['ㄷ','ㅏ']] ,[['ㅅ','ㅔ'],['']], # 어요 -> 다 
    [['ㅅ','ㅣ'],['ㄴ','ㅣ']] #시-> '' *
]

#종결 어미
EF = [
    [['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄷ','ㅏ']], #습니다 -> 다
    [[' ','ㅈ','ㅓ','ㄴ',' '],['ㄴ','ㅏ','ㄴ','ㅡ','ㄴ']],# 전 -> 나는
    [['ㅇ','ㅓ','ㅇ','ㅛ'],['ㅇ','ㅓ']], #어요 -> 어
    [['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ'],['ㄴ','ㄷ','ㅏ']],# ㅂ니다->ㄴ다
    [['ㅇ','ㅔ','ㅇ','ㅛ'],['ㅇ','ㅑ']], # 에요-> 야
    [['ㅇ','ㅖ','ㅇ','ㅛ'],['ㅇ','ㅑ']], #예요-> 야
    [['ㅇ','ㅛ'],['']], #요->''
    [['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['ㅇ','ㅓ']], #습니까 -> 어 *
    [['ㅂ','ㄴ','ㅣ','ㄲ','ㅏ'],['']] #ㅂ니까 ->'' *
    #,[['ㅅ','ㅔ'],['ㅝ'],['ㅘ']]
]

#대명사
NP = [
    
    [['ㅈ','ㅓ','ㄴ'],['ㄴ','ㅏ','ㄴ']],
    [['ㅈ','ㅓ'],['ㄴ','ㅏ']]
    
]

# 보조사
JX = [
   [['ㅇ','ㅣ','ㅇ','ㅛ'],['ㅇ','ㅣ','ㅇ','ㅑ']] 
]

#보조 용언
VX = [
    
    [['ㅈ','ㅜ'],['ㅈ','ㅜ'],['ㅈ','ㅝ']]
    
]

#동사
VV = [
    
    [['ㅇ','ㅗ'],['ㅇ','ㅗ'],['ㅇ','ㅘ']],
    [['ㅈ','ㅜ'],['ㅈ','ㅜ'],['ㅈ','ㅝ']],
    [['ㅎ','ㅏ'],['ㅎ','ㅏ'],['ㅎ','ㅐ']],
    
]
#형용사
VA = [
    #[['ㅃ','ㅡ'],['ㅃ','-'],['ㅃ','ㅡ','ㄴ','ㅣ']]
]

EXC_word = [
    
    ['ㅅㅔ',2]
    #['ㅅㅣ',2]
    
]

EXC_tags = [
    
    'EP',
    'EF'
    
]

XR = [
    [['ㅈ','ㅚ','ㅅ','ㅗ','ㅇ'],['ㅁ','ㅣ','ㅇ','ㅏ','ㄴ'] ]
]

class Jamodealer:
    jamo = []
    pp = ''
    #각 단어들을 받아와서 자모단위로 나눈다.
    def __init__(self,lis_word):
        
        #print('the jamo starts!')
        self.jamo = []
        for i in lis_word:
            self.jamo.append(split_syllables(i))
        
    def make_one(self):
        #list 형태로 저장된 자모들의 집합을 하나의 string pp에 저장한다. 
        self.pp = ''
        for i in self.jamo:
             self.pp= self.pp+i
        #자모 단위의 string에서 자모 단위로 사전을 만들고 거기에 index를 부여한다.        
        chars = list(set(self.pp))
        char_to_ix = { ch:i for i,ch in enumerate(chars) }
        ix_to_char = { i:ch for i,ch in enumerate(chars) }
        #자모 단위로 분리되었던 문장을 다시 하나로 합친다. 
        jamo_numbers = [char_to_ix[x] for x in self.pp]
        restored_jamo = ''.join([ix_to_char[x] for x in jamo_numbers])
        #합쳐진 문장을 return 한다.
        restored_text = join_jamos(restored_jamo)
        return restored_text

#list to string   
def li2str(input):
    st = ""
    for i in input:
        st = st+i
    return st

#string to list
def str2li(input):
    li = []
    for i in range(len(input)):
        li.append(input[i])
    return li

#To make Jamo Dictionary
#Dictionary의 요소를 jamo로 바꾸는 함수이다.
def makejamodict(input):
    result = []
    for i in input:
        bullet = []
        one = []
        two = []
        gre1 = tojamo(i[0])
        for j in gre1:
            for k in j:
                one.append(k)
        bullet.append(one)
        gre2 = tojamo(i[1])
        for j in gre2:
            for k in j:
                two.append(k)
        bullet.append(two)
        result.append(bullet)
    return result

#To make String dictionary
#Convert List element to String
#각 tag 사전의 key와 value가 string단위로 합쳐지게 해서 re.sub() 사용이 가능해진다.
def makestrdict(input):
    result = []
    for i in input:
        bullet = []
        for j in range(len(i)):
            gre = li2str(i[j])
            bullet.append(gre)
        result.append(bullet)
    return result


#추가한 사전에 대한 str 사전을 생성, mapping 시키는 부분

EP_dict = makestrdict(EP)
EF_dict = makestrdict(EF)
NP_dict = makestrdict(NP)
JX_dict = makestrdict(JX)
VX_dict = makestrdict(VX)
VV_dict = makestrdict(VV)
XR_dict = makestrdict(XR)
VA_dict = makestrdict(VA)
    
Dict_list=['EP','EF','NP','JX','VX','VV','VA','XR']

Dict_map = [EP_dict, EF_dict,NP_dict,JX_dict,VX_dict, VV_dict, VA_dict, XR_dict]


#divide by morpheme
#형태소로 변환 후 tag와 word를 따로 저장한다. 이때 각각의 index는 일치한다.
def to2lists(input):
    lis_word = []
    lis_tag = []
    #data = han.pos(input,ntags=22,flatten=True, join=False)
    data = mec.pos(input)
    for i in data:
        lis_word.append(i[0])
        lis_tag.append(i[1])
    return lis_word, lis_tag 

    
#add in 2021.09.26


class Changer(object):
    def high_low(self, stc):
        result = stc
        
        space_list = indee(stc,' ')
        
        lis_word, lis_tag = to2lists(result)
        space_location = ind_lili(space_list, lis_word)
        
        #split to Jamo(morphology)
        jam = Jamodealer(lis_word)
        lis = []
        #explore all tags in sentence
        for i in range(len(lis_tag)):
            res = jam.jamo[i]
            ##반말로 변환 가능성이 있는 tags
            for k in range(len(Dict_list)):
                if Dict_list[k] in lis_tag[i]:
                    dic = Dict_map[Dict_list.index(Dict_list[k])]
                    #res = jam.jamo[i]
                    for j in range(len(dic)):
                        #print(dic[j])\
                        #변환 가능한 element가 하나가 아닌 경우에 따라 달라질 수 있는 경우 
                        if self.isExcept(dic[j])==1:
                            ind = self.indicator(i,jam.jamo,lis_tag,EXC_word, EXC_tags)
                            res = re.sub(dic[j][0],dic[j][ind],res)
                        else:
                            res = re.sub(dic[j][0],dic[j][1],res)
                            
                    #jam.jamo[i] = res
            lis.append(res)
            
            
            #print(jam.jamo[i])
        union(space_location, lis)
        jam.jamo = []
        for i in range(len(lis)):
            jam.jamo.append(lis[i])
            #print(lis[i])
        
        #union(space_location, jam.jamo)
        
        return jam.make_one()
        
    def isExcept(self, input):
        if len(input)>=3:
            return 1
        else:
            return 0
        
    def indicator(self, ind, lis, tag, ex_word, exc_tags):
        re = 1 ## use 1 index element
        for j in range(len(ex_word)):
            #exception tags like EF, EP
            if exc_tags[j] in tag[ind+1]:
                for i in range(len(ex_word)):
                    #만약 예외 단어가 다음 단어와 일치한다면 바꾼다. 
                    if ex_word[i][0] in lis[ind+1]:
                        #use 2 index element
                        re = ex_word[i][1]
                        break
                        print(lis[ind-1])
        return re
        
    
    def processText(self,stc):
        result = stc
        res = self.high_low(result)
        #spacing = Spacing()
        #res = spacing(res)
        return res

def indee(lis, input):
    
    rlis = []
    
    for i in range(len(lis)):
        if lis[i]==input:
            rlis.append(i)
            
    for i in range(len(rlis)):
        rlis[i] = rlis[i]-i
            
    return rlis

def ind_lili(lis_space,lis_lis):
    
    rlis = []
    k=0
    for i in range(len(lis_lis)):
        
        if k in lis_space:
            rlis.append(i)
            
        k = k+len(lis_lis[i])
        
    
    return rlis

def union(lis, lis_lis):
    
    k = 0
    for i in lis:
        lis_lis.insert(i+k,' ')
        k = k+1



In [10]:
txt = '이거 하자.'
tx = '저는 뭐하십니까? 뭐하세요? 밥 먹었어요? 오십니까? 주십니까? 먹고 있으십니다. 생각을 해봤습니다. 오징어 게임에서 한 명만 살아남습니다. \
        계십니까? 저쪽으로 가세요. 바쁘십니까? 누구십니까? 오십니까? 안녕하십니까? 먹습니다. 먹으세요.' 
ch = Changer()
tt = ch.processText(txt)
ttt = ch.processText(tx)
print(tt)
print(ttt)

이거 하자.
나는 뭐하니? 뭐해? 밥 먹었어? 오니? 주니? 먹고 있으닌다. 생각을 해봤다. 오징어 게임에서 한 명만 살아남다. 계시? 나쪽으로 가. 바쁘니? 누구니? 오니? 안녕하니? 먹다. 먹으세.


In [11]:
import pandas as pd
number = 500
texts = pd.read_table('./data/hgu_clean.kr', sep ='\n')[:number]
texts = texts['26일이요?']

for i in range(0, number):
    tt = mec.pos(texts[i], flatten=True, join=True) 
    print(tt)

['27/SN', '일/NNBC', '은/JX', '괜찮/VA', '나요/EF', '?/SF']
['중단/NNG', '시켜서/XSV+EC', '죄송/XR', '해요/XSA+EF', './SF']
['중단/NNG', '시켜서/XSV+EC', '죄송/XR', '해요/XSA+EF', './SF']
['아/IC', ',/SC', '이런/MM', '!/SF', '도저히/MAG', '생각/NNG', '이/JKS', '안/MAG', '나/VV', '는/ETM', '걸/NNB+JKO', './SF']
['Joe/SL', "'/SY", 's/SL', 'Café/SL', '라는/VCP+ETM', '레스토랑/NNG', '에/JKB', '가/VV', '고/EC', '싶/VX', '은데/EC', ',/SC', '알/VV', '고/EC', '계세요/VX+EF', '?/SF']
['보다/MAG', '나/VA', '은/ETM', '건강/NNG', '과/JC', '생활/NNG', '회사/NNG', '의/JKG', '글렌/NNP', '토마스/NNP', '입니다/VCP+EF', './SF']
['스타워즈/NNP', '는/JX', '어때/VA+EF', '?/SF']
['데이터/NNG', '코디네이터/NNG', '나/JC', '데이터/NNG', '매니저/NNG', '가/JKS', '어떨까/VA+EC', '요/JX', '?/SF']
['화성/NNG', '에서/JKB', '온/VV+ETM', '로봇/NNG', '이/VCP', '요/EF', '!/SF', '좋/VA', '아요/EF', '!/SF', '특수/NNG', '효과/NNG', '가/JKS', '대단/XR', '하/XSA', '대요/EF', './SF']
['팩스/NNG', '를/JKO', '보내/VV+EC', '줘서/VX+EC', '감사/NNG', '합니다/XSV+EF', './SF']
['11/SN', '월/NNBC', '30/SN', '일/NNBC', '날짜/NNG', '로/JKB', '팩스/NNG', '를/JKO', '보내/VV+EC', 

['38/SN', '도/NNBC', '정도/NNG', '의/JKG', '열/NNG', '이/JKS', '있/VA', '어요/EF', './SF']
['3/SN', '교대/NNG', '로/JKB', '운영/NNG', '하/XSV', '는/ETM', '건/NNB+JX', '어떨까/VA+EC', '요/JX', '?/SF']
['3/SN', '년/NNBC', '전/NNG', '에/JKB', '재혼/NNG', '했/XSV+EP', '어요/EF', './SF']
['3/SN', '년/NNBC', '전이/NNG', '야/VCP+EF', './SF']
['3/SN', '년/NNBC', '만/NNB', '인가요/VCP+EF', '?/SF']
['3/SN', '년/NNBC', '전/NNG', '에/JKB', '위궤양/NNG', '으로/JKB', '입원/NNG', '했/XSV+EP', '습니다/EF', './SF']
['3/SN', '시/NNBC', '몇/MM', '분/NNBC', '전/NNG', '이/VCP', '에요/EF', './SF']
['3/SN', '시/NNBC', '5/SN', '분/NNBC', '전/NNG', '이/VCP', '에요/EF', './SF']
['3/SN', '시/NNBC', '가/JKS', '어때요/VA+EF', '?/SF']
['3/SN', '시/NNBC', '에서/JKB', '5/SN', '시/NNBC', '까지/JX', '시간/NNG', '이/JKS', '있/VA', '어요/EF', './SF']
['3/SN', '시/NNBC', '쯤/XSN', '에/JKB', '오/VV', '세요/EP+EF', './SF', '아무것/NP', '도/JX', '안/MAG', '가져와도/VV+EC', '되/VV', '요/EF', './SF', '구워/VV+EC', '먹/VV', '을/ETM', '음식/NNG', '이/JKS', '많이/MAG', '있/VA', '거든요/EF', './SF']
['3/SN', '일/NNBC', '빌리/VV', '고/EC', '싶/VX

['강도/NNG', '들/XSN', '이/JKS', '나와/VV+EC', '린다/NNP', '를/JKO', '강탈/NNG', '하/XSV', '려고/EC', '했/VX+EP', '어/EF', './SF', '그래서/MAJ', '힘껏/MAG', '빨리/MAG', '달렸/VV+EP', '지/EF', './SF']
['강/NNG', '에/JKB', '어린/VA+ETM', '여자/NNG', '아이/NNG', '가/JKS', '빠졌/VV+EP', '습니다/EF', './SF']
['강요/NNG', '하/XSV', '지/EC', '않/VX', '아도/EC', '돼/VV+EF', './SF']
['내/NP', '가/JKS', '얼마나/MAG', 'Boyz/SL', 'Ⅱ/SN', 'Men/SL', '을/JKO', '좋아하/VV', '는지/EC', '알/VV', '잖아/EF', './SF']
['강의/NNG', '중/NNB', '에/JKB', '는/JX', '별로/MAG', '발언/NNG', '하/XSV', '지/EC', '않/VX', '아요/EF', './SF']
['강하/VA', '게/EC', '해/VV+EC', '주/VX', '세요/EP+EF', './SF']
['강하/VA', '게/EC', '해/VV+EC', '주/VX', '세요/EP+EF', './SF']
['갖/VV', '고/EC', '싶/VX', '은/ETM', '것/NNB', '이/JKS', '보이/VV', '질/EC+JX', '않/VX', '습니다/EF', './SF']
['같/VA', '은/ETM', '실수/NNG', '를/JKO', '반복/NNG', '하/XSV', '는/ETM', '것/NNB', '같/VA', '아요/EF', './SF']
['같/VA', '은/ETM', '토양/NNG', '층/NNG', '에서/JKB', '동물/NNG', '의/JKG', '뼈/NNG', '와/JC', '불/NNG', '을/JKO', '사용/NNG', '한/XSV+ETM', '증거/NNG', '를/JKO', '발견/NNG

In [12]:
import pandas as pd
number = 200
texts = pd.read_table('./data/hgu_clean.kr', sep ='\n')[:number]
texts = texts['26일이요?']

ch = Changer()

for i in range(0, number):
    tt = ch.processText(texts[i])
    print(tt)

27일은 괜찮나?
중단시켜서 미안해.
중단시켜서 미안해.
아, 이런! 도저히 생각이 안 나는걸.
Joe's Café 라는 레스토랑에 가고 싶은데, 알고 계세?
보다 나은 건강과 생활 회사의 글렌 토마스인다.
스타워즈는 어때?
데이터 코디네이터나 데이터 매니저가 어떨까요?
화성에서 온 로봇이! 좋아! 특수 효과가 대단하대.
팩스를 보내줘서 감사한다.
11월30일 날짜로 팩스를 보내줘서 감사한다.
자격을 취득하기 위해 공부 중이야.
간호사 자격을 취득하기 위해 공부 중이야.
국가시험을 향해 공부 중이야.
공무원에 되기 위한 국가시험을 향해 공부 중이야.
잘 됐어. 그럼 이제 집에 가도 되나?
미안한다. 그녀는 안에 없네. 메모 남겨 드릴까?
내려 줄래.
올려 줄래.
작은 목소리
스미스 선생님. 저 화장실에 가야겠는데.
화장실이 정말 급해!
그럼 총수입의 얼마나 되는 거죠?
0136521798이야.
1,199달러야?
100그램에 비타민C의 함유량은 레몬보다 높다.
100미터 더 가서 좌회전해.
105번이 그 근처에 서는 것 같은데, 확실하진 않아.
낸시에게 물어 보. 낸시는 나보다 더 자주 그 버스를 이용하거든.
105편의 탑승게이트는 변경되었어?
10분 정도 후에 다시 걸어줘.
10분 후에 다시 걸죠.
10분 후에 와.
10분정도 기다리닐 수 있어?
10세 여자 아이의 선물로는 무엇이 좋을 것 같어?
10시 정각이야.
10시 정각이야.
10시쯤에 목욕해.
10일에는 바쁘니?
11번가와 브로드웨이가 만나는 모퉁이로 가는 버스가 있나?
123호실 좀 대 줘?
123호실이오.
125쪽인다.
12시부터 1시까지야.
12시쯤에 자요.
12일 월요일에 뉴욕에서 필라델피아로 가는 표를 구하려고 하는데.
12페이지 밖에 안 돼. 인쇄하는데 오래 걸리지 않을 거야.
14달러 47센트인다. 정말 미안한다.
15분 쯤 뒤에 내 방으로 와 주닐래?
15세이기때문에 술은 마실 수 없다.
15일이야.
17일은 몇 요일이야?
1981년에 태어났어.
1990년에 동화대학에 입학을 했어.
19